In [65]:
import pandas as pd



df = pd.read_csv('BackendIM_Keywords_with_intent.csv')
print(df.summary())

AttributeError: 'DataFrame' object has no attribute 'summary'

In [49]:
# Fill missing search volume with 0, convert to numeric
df['Avg. monthly searches'] = pd.to_numeric(df['Avg. monthly searches'], errors='coerce').fillna(0)
df.dropna(subset=['Keyword'], inplace=True)  # Drop rows with no keyword
print(f"Total rows after cleaning: {len(df)}")

Total rows after cleaning: 7772


In [ ]:
# Filter relevant keywords
relevant_terms = ['api', 'backend', 'development', 'test', 'deploy', 'agile']
relevant_df = df[df['Keyword'].str.contains('|'.join(relevant_terms), case=False, na=False)]
print(f"Filtered rows: {len(relevant_df)}")

Filtered rows: 6712


In [55]:
# Vectorize keywords
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(relevant_df['Keyword'])

In [56]:
# Cluster into 4 groups
kmeans = KMeans(n_clusters=4, random_state=42)
relevant_df.loc[:, 'Cluster'] = kmeans.fit_predict(X)

In [33]:
import pandas as pd

# Define a function to infer intent
def assign_intent(keyword):
    if 'how to' in keyword or 'process' in keyword or 'introduction' in keyword:
        return 'Informational'
    elif 'tools' in keyword or 'platform' in keyword or 'deploy' in keyword:
        return 'Transactional'
    elif 'best' in keyword or ('software' in keyword.lower() and 'tools' not in keyword):
        return 'Commercial'
    else:
        return 'Navigational'

# Remove duplicates (optional, if needed)
relevant_df = relevant_df.drop_duplicates(subset=['Keyword'])

# Apply intent labels
relevant_df.loc[:, 'Intent'] = relevant_df['Keyword'].apply(assign_intent)

# Group by intent and show top keywords
for intent, group in relevant_df.groupby('Intent'):
    print(f"\n{intent} Intent:")
    print(group[['Keyword', 'Avg. monthly searches']].sort_values('Avg. monthly searches', ascending=False).head(5))


Commercial Intent:
                                      Keyword  Avg. monthly searches
499             adaptive software development              5000000.0
126                software agile development               500000.0
464    agile software development methodology               500000.0
471  manifesto for agile software development                50000.0
210                    agile software meaning                50000.0

Informational Intent:
                                       Keyword  Avg. monthly searches
9                    agile development process                50000.0
10                   agile methodology process                50000.0
1005        software engineering agile process                 5000.0
718   agile process model software engineering                 5000.0
902          introduction to agile development                 5000.0

Navigational Intent:
                            Keyword  Avg. monthly searches
465        agile system development       

In [34]:
intent_summary = relevant_df.groupby('Intent').agg({
    'Keyword': 'count',
    'Avg. monthly searches': 'sum'
}).rename(columns={'Keyword': 'Count'})
print("\nIntent Summary:")
print(intent_summary)


Intent Summary:
               Count  Avg. monthly searches
Intent                                     
Commercial       417              7008100.0
Informational     70               147500.0
Navigational    4943              4751900.0
Transactional    315               197950.0


In [35]:
relevant_df.to_csv('clustered_keywords.csv', index=False)

In [47]:
# Define landing page templates
landing_pages = {
    'Informational': {
        'Title': "Master Agile Backend Development with Backend IM",
        'Subtitle': "Learn How to Streamline API Creation and Testing",
        'Description': "Discover how Backend IM empowers developers with an agile approach to backend development. Explore step-by-step guides on rapidly creating APIs, deploying them seamlessly, and testing endpoints—all designed to accelerate your workflow and deliver results faster.",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Informational'].head(5)['Keyword'].tolist()
    },
    'Transactional': {
        'Title': "Build and Test APIs Instantly with Backend IM",
        'Subtitle': "Your All-in-One Tool for Rapid Backend Development",
        'Description': "Backend IM is the ultimate tool for developers who need to create, deploy, and test APIs on the fly. With an interactive interface and seamless integration, you can iterate quickly and ship robust backend solutions without the hassle—start today!",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Transactional'].head(5)['Keyword'].tolist()
    },
    'Commercial': {
        'Title': "The Best Tool for Rapid Backend Development",
        'Subtitle': "Why Backend IM Outshines the Rest",
        'Description': "Looking for the top solution to streamline backend development? Backend IM offers unmatched speed and simplicity, letting you build, deploy, and test APIs effortlessly. Compare us to the competition and see why we’re the preferred choice for agile teams.",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Commercial'].head(5)['Keyword'].tolist()
    },
    'Navigational': {
        'Title': "Backend IM: Your Go-To Backend Development Platform",
        'Subtitle': "Faster APIs, Smarter Testing, Agile Results",
        'Description': "Welcome to Backend IM—the tool trusted by developers to create and deploy APIs in record time. Designed for agility, our platform offers instant endpoint testing and seamless integration. Join the revolution in backend development now!",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Navigational'].head(5)['Keyword'].tolist()
    }
}

# Print landing page proposals
for intent, page in landing_pages.items():
    print(f"\nLanding Page for {intent} Intent:")
    print(f"Title: {page['Title']}")
    print(f"Subtitle: {page['Subtitle']}")
    print(f"Description: {page['Description']}")
    print(f"Target Keywords: {', '.join(page['Keywords'])}")


Landing Page for Informational Intent:
Title: Master Agile Backend Development with Backend IM
Subtitle: Learn How to Streamline API Creation and Testing
Description: Discover how Backend IM empowers developers with an agile approach to backend development. Explore step-by-step guides on rapidly creating APIs, deploying them seamlessly, and testing endpoints—all designed to accelerate your workflow and deliver results faster.
Target Keywords: agile development process, agile methodology process, agile software development process, agile development process diagram, agile process in software engineering

Landing Page for Transactional Intent:
Title: Build and Test APIs Instantly with Backend IM
Subtitle: Your All-in-One Tool for Rapid Backend Development
Description: Backend IM is the ultimate tool for developers who need to create, deploy, and test APIs on the fly. With an interactive interface and seamless integration, you can iterate quickly and ship robust backend solutions without

In [ ]:
dm = pd.read_csv('clustered_keywords.csv')
dm.head()

,Keyword,Avg. monthly searches,Competition,Cluster,Intent
0,api deployment tools,50.0,Medium,0,Transactional
1,agile developer tools,50.0,Unknown,3,Transactional
2,api creation platform,50.0,High,0,Transactional
3,api developer,5000.0,Low,0,Navigational
4,postman api,50000.0,Low,0,Navigational


In [46]:
#find uqiue words under intent
print(dm['Cluster'].value_counts())

Cluster
0    2853
2    1450
3     978
1     464
Name: count, dtype: int64


In [57]:
# Define landing page templates with total search volume
landing_pages = {
    'Informational': {
        'Title': "Master Agile Backend Development with Backend IM",
        'Subtitle': "Learn How to Streamline API Creation and Testing",
        'Description': "Discover how Backend IM empowers developers with an agile approach to backend development. Explore step-by-step guides on rapidly creating APIs, deploying them seamlessly, and testing endpoints—all designed to accelerate your workflow and deliver results faster.",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Informational'].sort_values('Avg. monthly searches', ascending=False).head(3)['Keyword'].tolist(),
    },
    'Transactional': {
        'Title': "Build and Test APIs Instantly with Backend IM",
        'Subtitle': "Your All-in-One Tool for Rapid Backend Development",
        'Description': "Backend IM is the ultimate tool for developers who need to create, deploy, and test APIs on the fly. With an interactive interface and seamless integration, you can iterate quickly and ship robust backend solutions without the hassle—start today!",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Transactional'].sort_values('Avg. monthly searches', ascending=False).head(5)['Keyword'].tolist(),
    },
    'Commercial': {
        'Title': "The Best Tool for Rapid Backend Development",
        'Subtitle': "Why Backend IM Outshines the Rest",
        'Description': "Looking for the top solution to streamline backend development? Backend IM offers unmatched speed and simplicity, letting you build, deploy, and test APIs effortlessly. Compare us to the competition and see why we’re the preferred choice for agile teams.",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Commercial'].sort_values('Avg. monthly searches', ascending=False).head(5)['Keyword'].tolist(),
    },
    'Navigational': {
        'Title': "Backend IM: Your Go-To Backend Development Platform",
        'Subtitle': "Faster APIs, Smarter Testing, Agile Results",
        'Description': "Welcome to Backend IM—the tool trusted by developers to create and deploy APIs in record time. Designed for agility, our platform offers instant endpoint testing and seamless integration. Join the revolution in backend development now!",
        'Keywords': relevant_df[relevant_df['Intent'] == 'Navigational'].sort_values('Avg. monthly searches', ascending=False).head(5)['Keyword'].tolist(),
    }
}

# Calculate total search volume for each intent's keywords
for intent, page in landing_pages.items():
    # Get the subset of the DataFrame matching the keywords
    keywords_df = relevant_df[relevant_df['Keyword'].isin(page['Keywords'])]
    # Sum the Avg. monthly searches
    total_volume = keywords_df['Avg. monthly searches'].sum()
    # Add to the dictionary
    page['Total Search Volume'] = total_volume

# Print landing page proposals with total search volume
for intent, page in landing_pages.items():
    print(f"\nLanding Page for {intent} Intent:")
    print(f"Title: {page['Title']}")
    print(f"Subtitle: {page['Subtitle']}")
    print(f"Description: {page['Description']}")
    print(f"Target Keywords: {', '.join(page['Keywords'])}")
    print(f"Total Search Volume: {int(page['Total Search Volume']):,}")


Landing Page for Informational Intent:
Title: Master Agile Backend Development with Backend IM
Subtitle: Learn How to Streamline API Creation and Testing
Description: Discover how Backend IM empowers developers with an agile approach to backend development. Explore step-by-step guides on rapidly creating APIs, deploying them seamlessly, and testing endpoints—all designed to accelerate your workflow and deliver results faster.
Target Keywords: agile development process, agile methodology process, software engineering agile process
Total Search Volume: 105,000

Landing Page for Transactional Intent:
Title: Build and Test APIs Instantly with Backend IM
Subtitle: Your All-in-One Tool for Rapid Backend Development
Description: Backend IM is the ultimate tool for developers who need to create, deploy, and test APIs on the fly. With an interactive interface and seamless integration, you can iterate quickly and ship robust backend solutions without the hassle—start today!
Target Keywords: agi

In [58]:
for cluster in range(4):
    print(f"\nCluster {cluster}:")
    print(relevant_df[relevant_df['Cluster'] == cluster]['Keyword'].head(5))


Cluster 0:
41                    rest api development
89              rest api development tools
104    free online rest api for developers
105        rest api testing practice sites
123           rest api development company
Name: Keyword, dtype: object

Cluster 1:
453                    create backend apis fast
4069                           create rest apis
4096    create rest api for sql server database
4103         create api for sql server database
4114                     create an api endpoint
Name: Keyword, dtype: object

Cluster 2:
0     api deployment tools
1    agile developer tools
2    api creation platform
3            api developer
4              postman api
Name: Keyword, dtype: object

Cluster 3:
36          python api development
54            python api developer
130    api development with python
173      api development in python
230                api jira python
Name: Keyword, dtype: object
